In [1]:
!pip install python-dotenv

# Create .env file containing the following line:
# LASTMILE_API_TOKEN=<your-token>
# You can get your key from the "API Token" section of https://lastmileai.dev/settings?page=tokens 
import dotenv
import os
dotenv.load_dotenv()
lastmile_api_token = os.getenv("LASTMILE_API_TOKEN")

In [2]:
import requests
from requests import Response

# First create a project Id
project_name = "Give me a cool project name!"

lastmile_evaluation_projects_endpoint = (
    "https://lastmileai.dev/api/evaluation_projects/create"
)
response: Response = requests.post(
    lastmile_evaluation_projects_endpoint,
    headers={"Authorization": f"Bearer {lastmile_api_token}"},
    json={
        "name": project_name,
        # "testCases": [{"query": query_str}],
    },
    timeout=60,  # TODO: Remove hardcoding
)

# Save this projectId and pass it into the other two endpoints below
print("Results from evaluation_projects create endpoint:")
print(response.json())

# Result received was clv5g0a2n01oiquexme9izxr5

Results from evaluation_projects create endpoint:
{'id': 'clv5g0a2n01oiquexme9izxr5', 'createdAt': '2024-04-18T16:15:16.223Z', 'updatedAt': '2024-04-18T16:15:16.223Z', 'name': 'Give me a cool project name!', 'metadata': None, 'creatorId': 'clp1m7n3l0062qpqnd4nyabbl', 'organizationId': None, 'visibility': 'PRIVATE', 'active': True}


In [4]:
# Now create a testSetId

import json

import requests
from requests import Response

from lastmile_eval.rag.debugger.api import (
    ContextRetrieved,
    LLMOutputReceived,
    PromptResolved,
    QueryReceived,
)

# Mock some example data for a test Case
query = QueryReceived(query="What is Rossdan's favourite movie?")
context = ContextRetrieved(
    context=[
        "Rossdan is a movie buff.",
        "Rossdan has rated 8 movies and 2 TV shows 10/10 according to https://www.imdb.com/user/ur60976563/ratings?sort=your_rating,desc&ratingFilter=0&mode=detail&ref_=undefined&lastPosition=0",
    ]
)
fully_resolved_prompt = PromptResolved(
    fully_resolved_prompt=f"User Query: {query.query}\n\nContext: {context.context}"
)
output = LLMOutputReceived(
    llm_output="Rossdan's favourite movie is Inception."
)
ground_truth = {"answer": "Rossdan's favourite movie is In The Mood For Love"}

test_cases = [
    {
        "query": query.model_dump_json(),
        "context": context.model_dump_json(),
        "fullyResolvedPrompt": fully_resolved_prompt.model_dump_json(),
        "output": output.model_dump_json(),
        "groundTruth": json.dumps(ground_truth),
    }
]

lastmile_test_set_create_endpoint = (
    "https://lastmileai.dev/api/evaluation_test_sets/create"
)
response: Response = requests.post(
    lastmile_test_set_create_endpoint,
    headers={"Authorization": f"Bearer {lastmile_api_token}"},
    json={
        "name": "new_test_set",
        "testCases": test_cases,
        "projectId": "clv5g0a2n01oiquexme9izxr5",
        # TODO: Define optional fields below
        # "orgId": "clv4693s2002cpbpoxro5ogzr",
        # "metadata": context_retrieved,
        # "visibility": fully_resolved_prompt,
    },
    timeout=60,  # TODO: Remove hardcoding
)

# Save this testSetId and pass it into the evaluation_sets endpoint
print("Results from evaluation_test_sets create endpoint:")
print(response.json())

# Result received was clv5g1fcs009lqyhakxxh756b

Results from evaluation_test_sets create endpoint:
{'id': 'clv5g1fcs009lqyhakxxh756b', 'createdAt': '2024-04-18T16:16:09.723Z', 'updatedAt': '2024-04-18T16:16:09.723Z', 'name': 'new_test_set', 'description': None, 'creatorId': 'clp1m7n3l0062qpqnd4nyabbl', 'projectId': 'clv5g0a2n01oiquexme9izxr5', 'organizationId': None, 'visibility': 'MEMBER', 'active': True, 'metadata': None}


In [7]:
# Now that you now have both projectId and testSetId,
# we can now create an evaluation set
evaluation_set_name = "Favourite Movie Checker Evaluation Set"

lastmile_evaluation_sets_endpoint = (
    "https://lastmileai.dev/api/evaluation_sets/create"
)
response: Response = requests.post(
    lastmile_evaluation_sets_endpoint,
    headers={"Authorization": f"Bearer {lastmile_api_token}"},
    json={
        "name": evaluation_set_name,
        "testSetId": "clv5g1fcs009lqyhakxxh756b",  # Received from earlier cell
        "evaluationSetMetrics": [],  # TODO: Define this with metrics
        "evaluationMetrics": [],  # TODO: Define this with metric data
        "projectId": "clv5g0a2n01oiquexme9izxr5",  # Received from earlier cell
        # TODO: Add optional fields below
        # orgId
        # paramSet
        # metadata
        # visibility
    },
    timeout=60,  # TODO: Remove hardcoding
)
print("Results from evaluation_sets create endpoint:")
print(response.json())

# Result received was clv5g7qm5008lqptgsn12w8ss

Results from evaluation_sets create endpoint:
{'id': 'clv5g7qm5008lqptgsn12w8ss', 'createdAt': '2024-04-18T16:21:04.252Z', 'updatedAt': '2024-04-18T16:21:04.252Z', 'name': 'Favourite Movie Checker Evaluation Set', 'paramSet': None, 'testSetId': 'clv5g1fcs009lqyhakxxh756b', 'creatorId': 'clp1m7n3l0062qpqnd4nyabbl', 'projectId': 'clv5g0a2n01oiquexme9izxr5', 'organizationId': None, 'visibility': 'MEMBER', 'metadata': None, 'active': True}
